In [9]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score,
    mean_absolute_percentage_error
)
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# Define the Advanced Ensemble Regressor class
class AdvancedEnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, random_state=42):
        self.random_state = random_state
        
        # Base models with optimized hyperparameters
        self.base_models = [
            RandomForestRegressor(n_estimators=200, max_depth=30, random_state=self.random_state),
            ExtraTreesRegressor(n_estimators=150, max_depth=25, random_state=self.random_state),
            XGBRegressor(n_estimators=150, learning_rate=0.05, random_state=self.random_state)
        ]
        
        # Meta model
        self.meta_model = XGBRegressor(n_estimators=100, learning_rate=0.03, random_state=self.random_state)
        
        # Feature selector
        self.feature_selector = None
        
    def _select_features(self, X, y):
        # Use Random Forest to select important features
        selector = RandomForestRegressor(n_estimators=100, random_state=self.random_state)
        selector.fit(X, y)
        
        # Select features with importance > mean
        selection = SelectFromModel(selector, threshold='mean')
        selection.fit(X, y)
        
        return selection
    
    def fit(self, X, y):
        # Select important features
        print("Selecting important features...")
        self.feature_selector = self._select_features(X, y)
        X_selected = self.feature_selector.transform(X)
        print(f"Selected {X_selected.shape[1]} out of {X.shape[1]} features")
        
        # Train base models
        print("Training base models...")
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            print(f"Training base model {i+1}/{len(self.base_models)}...")
            # Train with selected features
            model.fit(X_selected, y)
            meta_features[:, i] = model.predict(X_selected)
        
        # Add original features to meta features
        extended_meta_features = np.hstack((meta_features, X_selected))
        
        # Train meta model
        print("Training meta model...")
        self.meta_model.fit(extended_meta_features, y)
        
        return self
    
    def predict(self, X):
        # Select features
        X_selected = self.feature_selector.transform(X)
        
        # Get base model predictions
        meta_features = np.column_stack([
            model.predict(X_selected) for model in self.base_models
        ])
        
        # Combine with selected original features
        extended_meta_features = np.hstack((meta_features, X_selected))
        
        # Make final prediction
        return self.meta_model.predict(extended_meta_features)

# Load the dataset (update this path to your actual file location)
df = pd.read_csv('C:/Users/adity/z class work/RIC/Brazil fipe_2022.csv')

# Prepare the data
inputs = df.drop(['fipe_code', 'authentication', 'avg_price_brl', 'age_years'], axis='columns')
y = df['avg_price_brl']

le_month_of_reference = LabelEncoder()
le_brand = LabelEncoder()
le_model = LabelEncoder()
le_fuel = LabelEncoder()
le_gear = LabelEncoder()

inputs['month_of_reference_n'] = le_month_of_reference.fit_transform(inputs['month_of_reference'])
inputs['brand_n'] = le_brand.fit_transform(inputs['brand'])
inputs['model_n'] = le_model.fit_transform(inputs['model'])
inputs['fuel_n'] = le_fuel.fit_transform(inputs['fuel'])
inputs['gear_n'] = le_gear.fit_transform(inputs['gear'])

X = inputs.drop(['month_of_reference', 'brand', 'model', 'fuel', 'gear'], axis='columns')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Bagging Regressor': BaggingRegressor(estimator=DecisionTreeRegressor(), random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Extra Trees Regressor': ExtraTreesRegressor(random_state=42),
    'XGB Regressor': XGBRegressor(random_state=42),
    'Hist Gradient Boosting': HistGradientBoostingRegressor(random_state=42),
    'Advanced Ensemble': AdvancedEnsembleRegressor(random_state=42)  # Our new model
}

# Store results
results = pd.DataFrame(columns=[
    'Model', 'Training Time (s)', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R² Score'
])
predictions = {}

# Train and evaluate models
for name, model in models.items():
    print(f"\nTraining {name}...")
    # Time the training process
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    training_time = time.time() - start_time
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    predictions[name] = y_pred
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results = pd.concat([results, pd.DataFrame({
        'Model': [name],
        'Training Time (s)': [training_time],
        'MSE': [mse],
        'RMSE': [rmse],
        'MAE': [mae],
        'MAPE': [mape],
        'R² Score': [r2]
    })], ignore_index=True)

# Sort results by R2 score (descending)
results = results.sort_values('R² Score', ascending=False).reset_index(drop=True)

# Print detailed results
print("\nDetailed Model Performance Comparison:")
print(results.to_string(index=False))

# Print model analysis
print("\nModel Performance Analysis:")
for index, row in results.iterrows():
    print(f"\n{row['Model']}:")
    print(f"  Training Time: {row['Training Time (s)']:.4f} seconds")
    print(f"  MSE: {row['MSE']:.4f}")
    print(f"  RMSE: {row['RMSE']:.4f}")
    print(f"  MAE: {row['MAE']:.4f}")
    print(f"  MAPE: {row['MAPE']:.4f}")
    print(f"  R² Score: {row['R² Score']:.4f}")


Training Random Forest...

Training Bagging Regressor...

Training Decision Tree...

Training Extra Trees Regressor...

Training XGB Regressor...

Training Hist Gradient Boosting...

Training Advanced Ensemble...
Selecting important features...
Selected 3 out of 8 features
Training base models...
Training base model 1/3...
Training base model 2/3...
Training base model 3/3...
Training meta model...

Detailed Model Performance Comparison:
                 Model  Training Time (s)          MSE         RMSE          MAE     MAPE  R² Score
         Random Forest          77.742604 1.925372e+08 13875.779096  3097.374055 0.026214  0.998039
     Bagging Regressor           8.235249 2.114143e+08 14540.093992  3192.561019 0.027136  0.997847
         Decision Tree           1.147954 2.619145e+08 16183.773631  3422.997115 0.029584  0.997333
 Extra Trees Regressor          57.303754 3.295940e+08 18154.723573  3491.265641 0.029174  0.996643
         XGB Regressor           1.598747 2.043769e+09 45

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score,
    mean_absolute_percentage_error
)
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# Define the OptimizedHybridEnsemble class
class OptimizedHybridEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, random_state=42):
        self.random_state = random_state
        
        # Use your best models as base models with optimized parameters
        self.rf = RandomForestRegressor(
            n_estimators=300,
            max_depth=40,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features='sqrt',
            bootstrap=True,
            random_state=self.random_state
        )
        
        self.et = ExtraTreesRegressor(
            n_estimators=200,
            max_depth=40,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features='sqrt',
            bootstrap=True,
            random_state=self.random_state
        )
        
        # Use estimator instead of base_estimator
        self.bg = BaggingRegressor(
            estimator=DecisionTreeRegressor(max_depth=40),
            n_estimators=150,
            max_samples=0.8,
            max_features=0.8,
            bootstrap=True,
            bootstrap_features=False,
            random_state=self.random_state
        )
        
        # Model weights for weighted averaging
        self.weights = None
    
    def fit(self, X, y):
        # Train base models
        print("Training Random Forest...")
        self.rf.fit(X, y)
        
        print("Training Extra Trees...")
        self.et.fit(X, y)
        
        print("Training Bagging Regressor...")
        self.bg.fit(X, y)
        
        # Set fixed weights based on previous results
        rf_weight = 0.50  # RF had the best performance
        et_weight = 0.30  # ET was good but not as good as RF
        bg_weight = 0.20  # BG was third best
        
        # Store weights
        self.weights = [rf_weight, et_weight, bg_weight]
        print(f"Model weights: {self.weights}")
        
        return self
    
    def predict(self, X):
        # Get predictions from all models
        rf_pred = self.rf.predict(X)
        et_pred = self.et.predict(X)
        bg_pred = self.bg.predict(X)
        
        # Create weighted average
        final_pred = (
            self.weights[0] * rf_pred + 
            self.weights[1] * et_pred + 
            self.weights[2] * bg_pred
        )
        
        return final_pred

# Define the TwoStageOptimizedEnsemble class
class TwoStageOptimizedEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, random_state=42):
        self.random_state = random_state
        
        # First stage - strong models with tweaked parameters
        self.rf1 = RandomForestRegressor(
            n_estimators=300, max_depth=40, min_samples_split=2, 
            bootstrap=True, oob_score=True, random_state=self.random_state
        )
        
        self.rf2 = RandomForestRegressor(
            n_estimators=300, max_depth=None, min_samples_split=5,
            bootstrap=True, max_features=0.7, random_state=self.random_state
        )
        
        self.et = ExtraTreesRegressor(
            n_estimators=200, max_depth=40, min_samples_split=2,
            bootstrap=True, random_state=self.random_state
        )
        
        # Use estimator instead of base_estimator
        self.bg = BaggingRegressor(
            estimator=DecisionTreeRegressor(max_depth=40),
            n_estimators=200, max_samples=0.8, max_features=0.8,
            bootstrap=True, random_state=self.random_state
        )
        
        # Second stage - blending model
        self.final_model = RandomForestRegressor(
            n_estimators=100, max_depth=20, random_state=self.random_state
        )
        
        # For validation/blending
        self.kf = KFold(n_splits=5, shuffle=True, random_state=self.random_state)
    
    def fit(self, X, y):
        # Train first-stage models
        print("Training first-stage models...")
        self.rf1.fit(X, y)
        self.rf2.fit(X, y)
        self.et.fit(X, y)
        self.bg.fit(X, y)
        
        # Generate blending features using out-of-fold predictions
        blend_features = np.zeros((X.shape[0], 4))
        
        for train_idx, valid_idx in self.kf.split(X):
            # Train temporary models on fold
            rf1_fold = clone(self.rf1).fit(X[train_idx], y[train_idx])
            rf2_fold = clone(self.rf2).fit(X[train_idx], y[train_idx])
            et_fold = clone(self.et).fit(X[train_idx], y[train_idx])
            bg_fold = clone(self.bg).fit(X[train_idx], y[train_idx])
            
            # Predict on validation fold
            blend_features[valid_idx, 0] = rf1_fold.predict(X[valid_idx])
            blend_features[valid_idx, 1] = rf2_fold.predict(X[valid_idx])
            blend_features[valid_idx, 2] = et_fold.predict(X[valid_idx])
            blend_features[valid_idx, 3] = bg_fold.predict(X[valid_idx])
        
        # Train second-stage model on all blended features
        print("Training second-stage model...")
        self.final_model.fit(blend_features, y)
        
        return self
    
    def predict(self, X):
        # Generate predictions from first-stage models
        blend_pred = np.column_stack([
            self.rf1.predict(X),
            self.rf2.predict(X),
            self.et.predict(X),
            self.bg.predict(X)
        ])
        
        # Make final prediction with second-stage model
        return self.final_model.predict(blend_pred)

# Define the CarPriceOptimizedEnsemble class
class CarPriceOptimizedEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, random_state=42):
        self.random_state = random_state
        
        # Main model - highly optimized Random Forest
        self.main_model = RandomForestRegressor(
            n_estimators=500,
            max_depth=50,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features=0.8,
            bootstrap=True,
            n_jobs=-1,  # Use all available cores
            random_state=self.random_state
        )
        
        # Specialized model for feature selection
        self.feature_selector = SelectFromModel(
            estimator=RandomForestRegressor(n_estimators=100, random_state=self.random_state),
            threshold='mean'
        )
        
        # Secondary models for specific price ranges
        self.high_price_model = RandomForestRegressor(
            n_estimators=200, 
            max_depth=30,
            random_state=self.random_state
        )
        
        self.medium_price_model = ExtraTreesRegressor(
            n_estimators=200,
            max_depth=40,
            random_state=self.random_state
        )
        
        self.low_price_model = RandomForestRegressor(
            n_estimators=200,
            max_depth=40,
            random_state=self.random_state
        )
        
        # Price thresholds for segmentation (to be set during training)
        self.high_threshold = None
        self.low_threshold = None
    
    def fit(self, X, y):
        # Determine price thresholds for different segments
        self.high_threshold = np.percentile(y, 75)
        self.low_threshold = np.percentile(y, 25)
        
        print(f"High price threshold: {self.high_threshold}")
        print(f"Low price threshold: {self.low_threshold}")
        
        # Select important features
        print("Selecting important features...")
        self.feature_selector.fit(X, y)
        X_selected = self.feature_selector.transform(X)
        print(f"Selected {X_selected.shape[1]} out of {X.shape[1]} features")
        
        # Split data by price range
        high_mask = y >= self.high_threshold
        low_mask = y <= self.low_threshold
        medium_mask = ~(high_mask | low_mask)
        
        # Train main model on all data
        print("Training main model...")
        self.main_model.fit(X_selected, y)
        
        # Train specialized models for different price ranges
        if np.sum(high_mask) > 0:
            print("Training high price model...")
            self.high_price_model.fit(X_selected[high_mask], y[high_mask])
        
        if np.sum(medium_mask) > 0:
            print("Training medium price model...")
            self.medium_price_model.fit(X_selected[medium_mask], y[medium_mask])
        
        if np.sum(low_mask) > 0:
            print("Training low price model...")
            self.low_price_model.fit(X_selected[low_mask], y[low_mask])
        
        return self
    
    def predict(self, X):
        # Select features
        X_selected = self.feature_selector.transform(X)
        
        # Get base predictions from main model
        base_predictions = self.main_model.predict(X_selected)
        
        # Initialize array for refined predictions
        refined_predictions = np.copy(base_predictions)
        
        # Identify which price range each prediction falls into
        high_mask = base_predictions >= self.high_threshold
        low_mask = base_predictions <= self.low_threshold
        medium_mask = ~(high_mask | low_mask)
        
        # Apply specialized models for each segment
        if np.sum(high_mask) > 0:
            refined_predictions[high_mask] = self.high_price_model.predict(X_selected[high_mask])
        
        if np.sum(medium_mask) > 0:
            refined_predictions[medium_mask] = self.medium_price_model.predict(X_selected[medium_mask])
        
        if np.sum(low_mask) > 0:
            refined_predictions[low_mask] = self.low_price_model.predict(X_selected[low_mask])
        
        # Blend original and refined predictions
        # More weight to specialized models, less to general model
        final_predictions = 0.3 * base_predictions + 0.7 * refined_predictions
        
        return final_predictions

# Main execution code
# Load the dataset (update path to match your file location)
print("Loading dataset...")
df = pd.read_csv('C:/Users/adity/z class work/RIC/Brazil fipe_2022.csv')  # Update path if needed

# Prepare the data
print("Preparing data...")
inputs = df.drop(['fipe_code', 'authentication', 'avg_price_brl', 'age_years'], axis='columns')
y = df['avg_price_brl']

le_month_of_reference = LabelEncoder()
le_brand = LabelEncoder()
le_model = LabelEncoder()
le_fuel = LabelEncoder()
le_gear = LabelEncoder()

inputs['month_of_reference_n'] = le_month_of_reference.fit_transform(inputs['month_of_reference'])
inputs['brand_n'] = le_brand.fit_transform(inputs['brand'])
inputs['model_n'] = le_model.fit_transform(inputs['model'])
inputs['fuel_n'] = le_fuel.fit_transform(inputs['fuel'])
inputs['gear_n'] = le_gear.fit_transform(inputs['gear'])

X = inputs.drop(['month_of_reference', 'brand', 'model', 'fuel', 'gear'], axis='columns')

# Split the data
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
print("Initializing models...")
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Bagging Regressor': BaggingRegressor(estimator=DecisionTreeRegressor(), random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Extra Trees Regressor': ExtraTreesRegressor(random_state=42),
    'XGB Regressor': XGBRegressor(random_state=42),
    'Hist Gradient Boosting': HistGradientBoostingRegressor(random_state=42),
    'Optimized Hybrid Ensemble': OptimizedHybridEnsemble(random_state=42),
    'Two-Stage Ensemble': TwoStageOptimizedEnsemble(random_state=42),
    'Car Price Ensemble': CarPriceOptimizedEnsemble(random_state=42)
}

# Store results
results = pd.DataFrame(columns=[
    'Model', 'Training Time (s)', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R² Score'
])
predictions = {}

# Train and evaluate models
print("Training and evaluating models...")
for name, model in models.items():
    print(f"\n{'-'*50}")
    print(f"Training {name}...")
    
    # Time the training process
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    training_time = time.time() - start_time
    
    print(f"{name} trained in {training_time:.2f} seconds")
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    predictions[name] = y_pred
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results = pd.concat([results, pd.DataFrame({
        'Model': [name],
        'Training Time (s)': [training_time],
        'MSE': [mse],
        'RMSE': [rmse],
        'MAE': [mae],
        'MAPE': [mape],
        'R² Score': [r2]
    })], ignore_index=True)

# Sort results by R2 score (descending)
results = results.sort_values('R² Score', ascending=False).reset_index(drop=True)

# Print detailed results
print("\nDetailed Model Performance Comparison:")
print(results.to_string(index=False))

# Print model analysis
print("\nModel Performance Analysis:")
for index, row in results.iterrows():
    print(f"\n{row['Model']}:")
    print(f"  Training Time: {row['Training Time (s)']:.4f} seconds")
    print(f"  MSE: {row['MSE']:.4f}")
    print(f"  RMSE: {row['RMSE']:.4f}")
    print(f"  MAE: {row['MAE']:.4f}")
    print(f"  MAPE: {row['MAPE']:.4f}")
    print(f"  R² Score: {row['R² Score']:.4f}")

# Create visualization for the comparison
plt.figure(figsize=(12, 8))
plt.barh(results['Model'], results['R² Score'], color='skyblue')
plt.xlabel('R² Score')
plt.title('Model Performance Comparison')
plt.xlim(0.95, 1.0)  # Adjust based on your results
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.show()

# Create visualization for errors
plt.figure(figsize=(12, 8))
plt.barh(results['Model'], results['RMSE'], color='salmon')
plt.xlabel('RMSE (Lower is better)')
plt.title('Model Error Comparison')
plt.tight_layout()
plt.savefig('model_errors.png')
plt.show()

Loading dataset...
Preparing data...
Splitting data...
Scaling features...
Initializing models...
Training and evaluating models...

--------------------------------------------------
Training Random Forest...
Random Forest trained in 54.63 seconds

--------------------------------------------------
Training Bagging Regressor...
Bagging Regressor trained in 5.97 seconds

--------------------------------------------------
Training Decision Tree...
Decision Tree trained in 0.77 seconds

--------------------------------------------------
Training Extra Trees Regressor...
Extra Trees Regressor trained in 41.62 seconds

--------------------------------------------------
Training XGB Regressor...
XGB Regressor trained in 0.52 seconds

--------------------------------------------------
Training Hist Gradient Boosting...
Hist Gradient Boosting trained in 1.13 seconds

--------------------------------------------------
Training Optimized Hybrid Ensemble...
Training Random Forest...
Training Ext

KeyError: '[0, 16, 26, 30, 53, 60, 79, 80, 87, 91, 96, 106, 110, 139, 155, 172, 204, 239, 254, 282, 287, 288, 289, 297, 333, 338, 354, 358, 376, 386, 396, 397, 425, 427, 438, 445, 452, 459, 460, 465, 472, 494, 499, 501, 502, 517, 523, 524, 544, 567, 569, 578, 595, 602, 612, 631, 644, 666, 682, 705, 715, 730, 731, 746, 751, 758, 776, 779, 803, 818, 821, 831, 852, 856, 869, 885, 889, 919, 931, 939, 963, 975, 989, 990, 1012, 1014, 1054, 1057, 1077, 1089, 1091, 1105, 1110, 1111, 1128, 1129, 1131, 1151, 1159, 1166, 1174, 1219, 1226, 1230, 1233, 1238, 1249, 1253, 1274, 1278, 1295, 1305, 1309, 1326, 1349, 1351, 1368, 1374, 1399, 1403, 1407, 1413, 1421, 1427, 1435, 1443, 1445, 1446, 1451, 1462, 1470, 1488, 1493, 1502, 1509, 1516, 1522, 1541, 1547, 1581, 1592, 1603, 1612, 1617, 1619, 1624, 1626, 1637, 1645, 1647, 1656, 1674, 1681, 1683, 1704, 1708, 1710, 1715, 1730, 1731, 1744, 1747, 1763, 1770, 1776, 1786, 1807, 1821, 1827, 1828, 1846, 1848, 1870, 1871, 1877, 1885, 1904, 1909, 1914, 1951, 1956